In [ ]:
# 爬取10頁與韓國瑜有關的PTT標題與文章內容，並輸出成文件檔
import requests
from bs4 import BeautifulSoup

from urllib.parse import urljoin
base_url = 'https://www.ptt.cc/bbs/Gossiping/'

for page in range(1, 11):
#     搜尋條件與頁數作為參數加進url後，使用requests.get取得內容
    res = requests.get('https://www.ptt.cc/bbs/Gossiping/search?q=韓國瑜&page={}'.format(page))
    soup = BeautifulSoup(res.text, 'html.parser')
    
#     自動填寫滿18歲
    res = requests.post(res.url, data={'yes': 'yes'})

#     定義輸出的標題
    output_content = [['title', 'content']]
    soup = BeautifulSoup(res.text, 'html.parser')
    for post in soup.find_all('div', {"class": "title"}):
        print(post.a.string)
        if post.a:
#             取得文章標題
            post_title = post.a.string
            post_url = urljoin(base_url, post.a['href'])
            res = requests.get(post_url)
            res = requests.post(res.url, data={'yes': 'yes'})
            soup = BeautifulSoup(res.text, 'html.parser')
#             爬取文章內容
            content = soup.find('div', {'id': 'main-container'})
            cleaned_content = ''.join(content.text.split('\n\n')[1:]).split('--')[0]
#         將文章標題與內容放進輸出列表中
            output_content.append([post_title, cleaned_content])

In [ ]:
# 將爬取的PTT標題輸出成CSV
import csv
with open('ptt_titile.csv', 'w', encoding='utf-8-sig') as content:
    writer = csv.writer(content)
    writer.writerows(output_content)

In [ ]:
# 爬取中時軍事版圖片
import requests
from bs4 import BeautifulSoup

from urllib.parse import urljoin

url = 'https://www.chinatimes.com/armament/total?page=1'
r = requests.get(url)
# print(r.text)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())


In [ ]:
# 創立名為軍事的資料夾，將圖片爬取下來存於該資料夾中
import os
dir_path = "軍事"
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

count = 0
for img_tag in soup.find_all('img', {'class': 'photo'}):
    img_data = requests.get(urljoin(url, img_tag['src'])).content
    img_name = "image{}.jpg".format(count)
    
    with open(os.path.join(dir_path, img_name), 'wb') as img:
        img.write(img_data)
    count += 1
    print("{} images downloaded".format(count))

In [ ]:
# 爬取ETODAY軍事版圖片
import requests
from bs4 import BeautifulSoup

from urllib.parse import urljoin

import os
dir_path = "ETODAY軍事"

# 如果沒有名為ETODAY軍事的資料夾，就創立一個
if not os.path.exists(dir_path):
    os.mkdir(dir_path)


url = 'https://www.ettoday.net/news/focus/%E8%BB%8D%E6%AD%A6/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
# 定義下載圖片張數的計數器
count = 0

# 爬取每一篇新聞所在的tag
for post in soup.find_all('div', {'class': 'piece clearfix'}):
#     爬取每一篇新聞tag的標題
    title = post.find('h3').string
#     爬取每一篇新聞圖片的連結
    image_link = post.find('img').get('data-original')
    
#     如果有圖片連結，就將它的host拼接上，並載下來
    if image_link:
        image_url = urljoin(url, image_link)
        
        img_data = requests.get(image_url).content
#         圖片檔依照文章標題名取名
        img_name = "{}.jpg".format(title)
        
        with open(os.path.join(dir_path, img_name), 'wb') as img:
            img.write(img_data)
        
        count += 1
        print("{} images downloaded".format(count))